In [1]:
import findspark
findspark.init("D:/spark")

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext("local[4]", "MeslekKazanc")

In [4]:
simple_data_rdd = sc.textFile("D:/Datasets/simple_data.csv").filter(lambda x: "sirano" not in x)

In [5]:
simple_data_rdd.take(3)

['1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200',
 '3,Timur,30,Müzisyen,Istanbul,9000']

In [35]:
def impute_meslek(line):
    sirano = line.split(",")[0]
    isim = line.split(",")[1]
    yas = line.split(",")[2]
    meslek = line.split(",")[3]
    sehir = line.split(",")[4]
    maas = line.split(",")[5]
    if meslek == '':
        return sirano, isim, yas, 'Unknown', sehir, maas 
    else:
        return sirano, isim, yas, meslek, sehir, maas

In [36]:
simple_data_rdd_imputed = simple_data_rdd.map(impute_meslek)
simple_data_rdd_imputed.take(20)

[('1', 'Cemal', '35', 'Isci', 'Ankara', '3500'),
 ('2', 'Ceyda', '42', 'Memur', 'Kayseri', '4200'),
 ('3', 'Timur', '30', 'Müzisyen', 'Istanbul', '9000'),
 ('4', 'Burcu', '29', 'Pazarlamaci', 'Ankara', '4200'),
 ('5', 'Yasemin', '23', 'Unknown', 'Bursa', '4800'),
 ('6', 'Ali', '33', 'Memur', 'Ankara', '4250'),
 ('7', 'Dilek', '29', 'Pazarlamaci', 'Istanbul', '7300'),
 ('8', 'Murat', '31', 'Müzisyen', 'Istanbul', '12000'),
 ('9', 'Ahmet', '33', 'Doktor', 'Ankara', '18000'),
 ('10', 'Muhittin', '46', 'Berber', 'Istanbul', '12000'),
 ('11', 'Hicaziye', '47', 'Tuhafiyeci', 'Ankara', '4800'),
 ('12', 'Harun', '43', 'Tornacı', 'Ankara', '4200'),
 ('13', 'Hakkı', '33', 'Memur', 'Çorum', '3750'),
 ('14', 'Gülizar', '37', 'Doktor', 'İzmir', '14250'),
 ('15', 'Şehmuz', '41', 'Unknown', 'Ankara', '8700'),
 ('16', 'Gençay', '46', 'Berber', 'Ankara', '8800'),
 ('16', 'Gençay', '46', 'Berber', 'Ankara', '8800')]

In [37]:
#artık split etmeye gerek yok çünkü satırlar tuple oldu
def take_meslek_kazanc(line):
    meslek = line[3]
    kazanc = float(line[-1])
    
    return (meslek, kazanc)

In [38]:
meslek_kazanc_rdd = simple_data_rdd_imputed.map(take_meslek_kazanc)

In [39]:
meslek_kazanc_rdd.take(3)

[('Isci', 3500.0), ('Memur', 4200.0), ('Müzisyen', 9000.0)]

In [40]:
meslek_kazanc_rdd_rbk = meslek_kazanc_rdd.reduceByKey(lambda x,y: x+y)
meslek_kazanc_rdd_rbk.take(20)

[('Memur', 12200.0),
 ('Pazarlamaci', 11500.0),
 ('Unknown', 13500.0),
 ('Tuhafiyeci', 4800.0),
 ('Tornacı', 4200.0),
 ('Isci', 3500.0),
 ('Müzisyen', 21000.0),
 ('Doktor', 32250.0),
 ('Berber', 29600.0)]

In [41]:
# Geliri yüksek olandan düşük olana doğru sıralayınız.
meslek_kazanc_rdd_rbk.sortBy(lambda x: x[1], ascending=False).take(10)

[('Doktor', 32250.0),
 ('Berber', 29600.0),
 ('Müzisyen', 21000.0),
 ('Unknown', 13500.0),
 ('Memur', 12200.0),
 ('Pazarlamaci', 11500.0),
 ('Tuhafiyeci', 4800.0),
 ('Tornacı', 4200.0),
 ('Isci', 3500.0)]